In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.chrome.options import Options
import time
import os
import zipfile
from os import listdir
from os.path import isfile, join
from urllib.request import urlretrieve


chromedriver = "/Users/justinblinder/dev/drivers/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

'''instantiate a chrome options object so you can set the size and headless preference'''
chrome_options = Options()
chrome_options.add_argument("--headless")

''' add missing support for chrome "send_command"  to selenium webdriver (https://stackoverflow.com/a/47366981/190155)'''
#driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
#params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': '/Users/justinblinder/Downloads/srt/'}}
#command_result = driver.execute("send_command", params)

driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
#driver = webdriver.Chrome(chromedriver)
driver.get("https://subscene.com/")

# set wait time
driver.implicitly_wait(4) 

In [2]:
'''Collect a download url for each film'''
# Chrome headless mode does not allow dowloading files
# Cannot curl file url because it's a cloudflare redirect

download_path = '/Users/justinblinder/Downloads/'
download_links = {}

'''Main subtitle search page'''
def search_film(film_title):
    form_input = driver.find_element_by_name("q")
    form_input.send_keys(film_title)
    '''Search for film'''
    form_input.send_keys(Keys.RETURN)
    get_filtered_film(film_title)

def get_filtered_film(film_title):
    '''Similar film list page'''
    link = driver.find_element_by_xpath('//h2[text()="Exact"]/following-sibling::ul/descendant::div[@class="title"]/descendant::a')
    link.click()
    search_subtitles(film_title)


def check_exists_by_xpath(el,xpath):
    try:
        el.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

'''Individual film subtitle list page'''

"""Find all engish subtitles that have a quality flag and aren't for hearing impaired"""
def search_subtitles(film_title):
    english_selector = '//a[contains(@href,"english")]'
    subtitle_links = driver.find_elements_by_xpath(english_selector)
    valid_links = []
    for link in subtitle_links:
        is_quality = check_exists_by_xpath(link,'span[contains(@class,"positive-icon")]')
        hearing_impaired_el = link.find_element_by_xpath('../following-sibling::td/following-sibling::td')     
        is_hearing_impaired = False if hearing_impaired_el.get_attribute('class') == "a40" else True
        if is_quality is True and is_hearing_impaired is False:
            valid_links.append(link)
    valid_links[0].click()
    get_download_link(film_title)
    
'''Find and click download link'''
def get_download_link(film_title):
    download_button = driver.find_element_by_id('downloadButton')
    # TODO: if link is zip then click, else go back to revious page and get a new one
    download_links[film_title] = download_button.get_attribute('href')
    download_button.click()
    unzip_file(film_title)

''' Locally unzip file'''
def unzip_file(film_title):
    time.sleep(2)
    for file in os.listdir(download_path):
        if file.endswith(".zip"):
            filename = os.path.join(download_path, file)
            unzip(filename,film_title)
    driver.get("https://subscene.com/")
    
''' Rename unzipped srt file / zip file and mov to seperate folders'''
def unzip(filename,film_title):
    # unzip file to srt dir
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall(download_path)
    zip_ref.close()

    # move zip to archive folder
    os.rename(filename, download_path+"archive/"+film_title.replace(' ','-').replace(':','')+'.zip')

    # get unzipped srt file
    srtfile = None
    for file in os.listdir(download_path):
        if file.endswith(".srt"):
            srtfile = os.path.join(download_path, file)
            os.rename(srtfile , download_path + "srt/"+film_title.replace(' ','-').replace(':','')+'.srt')


        
films = ["almost famous", 'black panther','Star Wars: The Last Jedi','Rogue One']
for film in films:
    print(film)
    film_title = film
    search_film(film_title)
    
print(download_links)



almost famous
black panther
Star Wars: The Last Jedi
Rogue One
{'almost famous': 'https://subscene.com/subtitles/english-text/aXOOPLmznQbSR_7DfxyqWlzal1dwtVpfkvKAmh2NOVLiWfJb30OU4o6dBTmOebfTTHdKQ7Z2e1mK4Sg8GK-3OQ__iiDlQsRZrHFjLhv1rxFzSY98FcQ30RsUrJ46QI1s0', 'black panther': 'https://subscene.com/subtitles/english-text/7EkrqvM5HhCPcIjGR36JDiRhpfmGUgUCEgj8FmNaFXe3czLC4cC0JomNe5oflPmJDUtzodsEZq-dbPxQTCPUWiEdGkIhIdi_214HTvHBCWaApcPa42o9IEtjz4gL118Z0', 'Star Wars: The Last Jedi': 'https://subscene.com/subtitles/english-text/d7tGza9-ItfwzecaVzt1ZOXuCfgg27G9nh7a5Q-2zAl2TzEZnaLsxu3-y9-kNOHSn1aPsveWYnWUkQlT67hGwJsye9IF-Jbe68H5GRQFeRNvknZv217wjLn0nqPHhj940', 'Rogue One': 'https://subscene.com/subtitles/english-text/-7CO17-GnAZaB2Ew5TfMbnJ9vktQnwjgl4NtqpBFBeoDN8QmEfa5zeDgpRI7o3VvxuNgIWSSkY77jh1E8vOXR0L2KNNjOWkJzcXy5Jm5sliEy49Cjz4J3nTxhqReCnMm0'}


In [8]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://subscene.com/")
time.sleep(10) # wait until cloudflare DDoS protect stops
for film_title, url in download_links.items():
    driver.get(url)

TimeoutException: Message: timeout
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.13.4 x86_64)


In [93]:
#for film,url in download_links.items():
#    clean_title = film.replace(' ','-').replace(':','')
#    fullfilename = os.path.join('/Users/justinblinder/Downloads/srt/', clean_title + '.zip')
#    urlretrieve(url, fullfilename)